In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import joblib
import pandas as pd
import numpy as np
import os
from sklearn.metrics.pairwise import cosine_similarity
import time
os.chdir('/content/drive/MyDrive/product_recommendation')

In [3]:
!ls

aaic_task_data.csv  create_db.ipynb  glove_840B_300d  my_data.db  pickle_files


In [ ]:
df = joblib.load('16k_apperal_data_preprocessed')

In [ ]:
df

,asin,brand,color,medium_image_url,product_type_name,title,formatted_price
4,B004GSI2OS,FeatherLite,Onyx Black/ Stone,https://images-na.ssl-images-amazon.com/images...,SHIRT,featherlite ladies long sleeve stain resistant...,$26.26
6,B012YX2ZPI,HX-Kingdom Fashion T-shirts,White,https://images-na.ssl-images-amazon.com/images...,SHIRT,womens unique 100 cotton special olympics wor...,$9.99
15,B003BSRPB0,FeatherLite,White,https://images-na.ssl-images-amazon.com/images...,SHIRT,featherlite ladies moisture free mesh sport sh...,$20.54
27,B014ICEJ1Q,FNC7C,Purple,https://images-na.ssl-images-amazon.com/images...,SHIRT,supernatural chibis sam dean castiel neck tshi...,$7.39
46,B01NACPBG2,Fifth Degree,Black,https://images-na.ssl-images-amazon.com/images...,SHIRT,fifth degree womens gold foil graphic tees jun...,$6.95
...,...,...,...,...,...,...,...
183081,B01MRV2IFS,YueLian,Black,https://images-na.ssl-images-amazon.com/images...,SHIRT,yuelian womens chiffon short sleeves sun prote...,$19.25
183092,B01LY4QWLF,Vintage America,White,https://images-na.ssl-images-amazon.com/images...,SHIRT,vintage america womens large lace collared blo...,$23.24
183096,B07167SCNH,Tart Collections,Black,https://images-na.ssl-images-amazon.com/images...,SHIRT,tart womens collections ann wrap top xs black,$29.99
183101,B07575N2WX,Soprano,Gray,https://images-na.ssl-images-amazon.com/images...,SHIRT,soprano womens small tiefringe slubknit tank t...,$22.83


In [ ]:
df['formatted_price'] = df['formatted_price'].str.replace('$','')

In [ ]:
df['formatted_price'] = df['formatted_price'].str.strip()

In [ ]:
df['formatted_price'] = df['formatted_price'].apply(pd.to_numeric,errors='coerce')

In [ ]:
df.dtypes

asin                  object
brand                 object
color                 object
medium_image_url      object
product_type_name     object
title                 object
formatted_price      float64
dtype: object

In [ ]:
df.to_csv('aaic_task_data.csv',index=False)

In [5]:
dff = pd.read_csv('aaic_task_data.csv')
dff

,asin,brand,color,medium_image_url,product_type_name,title,formatted_price
0,B004GSI2OS,FeatherLite,Onyx Black/ Stone,https://images-na.ssl-images-amazon.com/images...,SHIRT,featherlite ladies long sleeve stain resistant...,26.26
1,B012YX2ZPI,HX-Kingdom Fashion T-shirts,White,https://images-na.ssl-images-amazon.com/images...,SHIRT,womens unique 100 cotton special olympics wor...,9.99
2,B003BSRPB0,FeatherLite,White,https://images-na.ssl-images-amazon.com/images...,SHIRT,featherlite ladies moisture free mesh sport sh...,20.54
3,B014ICEJ1Q,FNC7C,Purple,https://images-na.ssl-images-amazon.com/images...,SHIRT,supernatural chibis sam dean castiel neck tshi...,7.39
4,B01NACPBG2,Fifth Degree,Black,https://images-na.ssl-images-amazon.com/images...,SHIRT,fifth degree womens gold foil graphic tees jun...,6.95
...,...,...,...,...,...,...,...
16037,B01MRV2IFS,YueLian,Black,https://images-na.ssl-images-amazon.com/images...,SHIRT,yuelian womens chiffon short sleeves sun prote...,19.25
16038,B01LY4QWLF,Vintage America,White,https://images-na.ssl-images-amazon.com/images...,SHIRT,vintage america womens large lace collared blo...,23.24
16039,B07167SCNH,Tart Collections,Black,https://images-na.ssl-images-amazon.com/images...,SHIRT,tart womens collections ann wrap top xs black,29.99
16040,B07575N2WX,Soprano,Gray,https://images-na.ssl-images-amazon.com/images...,SHIRT,soprano womens small tiefringe slubknit tank t...,22.83


In [ ]:
dff['product_type_name'].value_counts()

SHIRT                               12827
BOOKS_1973_AND_LATER                 1334
APPAREL                               620
ACCESSORY                             302
SPORTING_GOODS                        194
DRESS                                 141
BLAZER                                 93
SWEATER                                79
OUTERWEAR                              69
OUTDOOR_RECREATION_PRODUCT             53
UNDERWEAR                              35
SLEEPWEAR                              34
MISC_OTHER                             33
HEALTH_PERSONAL_CARE                   33
PANTS                                  22
SKIRT                                  20
AUTO_ACCESSORY                         20
TOYS_AND_GAMES                         20
ETHNIC_WEAR                            16
HANDBAG                                 9
POWERSPORTS_PROTECTIVE_GEAR             9
BRA                                     8
BEAUTY                                  6
SHOES                             

In [ ]:
import sqlite3
conn = sqlite3.connect('my_data.db')
c = conn.cursor()

In [ ]:
c.execute('''DROP TABLE IF EXISTS aaic;''')

c.execute('''CREATE TABLE aaic(
    id INTEGER PRIMARY KEY,
    asin VARCHAR(15),
    brand VARCHAR(100),
    color VARCHAR(20),
    medium_image_url TEXT,
    product_type_name VARCHAR(20),
    title TEXT,
    formatted_price VARCHAR(5));''')

In [ ]:
dff.to_sql('aaic', conn, if_exists='append', index = False)

In [ ]:
pd.read_sql('''SELECT * FROM aaic ''', conn)

,id,asin,brand,color,medium_image_url,product_type_name,title,formatted_price
0,1,B004GSI2OS,FeatherLite,Onyx Black/ Stone,https://images-na.ssl-images-amazon.com/images...,SHIRT,featherlite ladies long sleeve stain resistant...,26.26
1,2,B012YX2ZPI,HX-Kingdom Fashion T-shirts,White,https://images-na.ssl-images-amazon.com/images...,SHIRT,womens unique 100 cotton special olympics wor...,9.99
2,3,B003BSRPB0,FeatherLite,White,https://images-na.ssl-images-amazon.com/images...,SHIRT,featherlite ladies moisture free mesh sport sh...,20.54
3,4,B014ICEJ1Q,FNC7C,Purple,https://images-na.ssl-images-amazon.com/images...,SHIRT,supernatural chibis sam dean castiel neck tshi...,7.39
4,5,B01NACPBG2,Fifth Degree,Black,https://images-na.ssl-images-amazon.com/images...,SHIRT,fifth degree womens gold foil graphic tees jun...,6.95
...,...,...,...,...,...,...,...,...
16037,16038,B01MRV2IFS,YueLian,Black,https://images-na.ssl-images-amazon.com/images...,SHIRT,yuelian womens chiffon short sleeves sun prote...,19.25
16038,16039,B01LY4QWLF,Vintage America,White,https://images-na.ssl-images-amazon.com/images...,SHIRT,vintage america womens large lace collared blo...,23.24
16039,16040,B07167SCNH,Tart Collections,Black,https://images-na.ssl-images-amazon.com/images...,SHIRT,tart womens collections ann wrap top xs black,29.99
16040,16041,B07575N2WX,Soprano,Gray,https://images-na.ssl-images-amazon.com/images...,SHIRT,soprano womens small tiefringe slubknit tank t...,22.83


In [ ]:
def get_db_connection():
    conn = sqlite3.connect('my_data.db')
    conn.row_factory = sqlite3.Row
    return conn

In [ ]:
con = get_db_connection()

In [ ]:
post = con.execute('SELECT * FROM aaic ORDER BY formatted_price DESC LIMIT 20').fetchall()
post[0]['asin']

'B01IO1GL4U'

In [ ]:
pd.read_sql_query('''SELECT * from aaic ORDER BY formatted_price DESC LIMIT 20''',conn)

,id,asin,brand,color,medium_image_url,product_type_name,title,formatted_price
0,13055,B01IO1GL4U,Brunello Cucinelli,Gray,https://images-na.ssl-images-amazon.com/images...,SHIRT,brunello cucinelli womens gray satin silk blaz...,995.0
1,1366,B01LBQC1IU,Maison Martin Margiela,Multi-color,https://images-na.ssl-images-amazon.com/images...,SHIRT,maison martin margiela mm6 womens sleeveless b...,99.99
2,1643,B01L7OJBGG,Maison Martin Margiela,Burgundy,https://images-na.ssl-images-amazon.com/images...,SHIRT,maison martin margiela 1 womens burgundy cap s...,99.99
3,2051,B01FUYK5QO,Dolce & Gabbana,Black/ivory,https://images-na.ssl-images-amazon.com/images...,SHIRT,dolce gabbana black ivory striped cotton shor...,99.99
4,2387,B06XSHXWK5,HUGO BOSS,Multi-color,https://images-na.ssl-images-amazon.com/images...,SHIRT,boss esabia windowpane print twist detail jers...,99.99
5,2458,B071JQJFWR,KingYuan,Flesh,https://images-na.ssl-images-amazon.com/images...,SHIRT,kingyuan 105d 42e 46d 48c wider tear drop ov...,99.99
6,2587,B01N8Z2K60,James Perse,Black,https://images-na.ssl-images-amazon.com/images...,SHIRT,james perse black shirt 1,99.99
7,2800,B01FUTECN6,Dolce & Gabbana,Blue/White,https://images-na.ssl-images-amazon.com/images...,SHIRT,dolce gabbana blue white striped cotton cap s...,99.99
8,3811,B01EUK0EDS,Ralph Lauren Black Label,Black,https://images-na.ssl-images-amazon.com/images...,SHIRT,ralph lauren black label collection denim blac...,99.99
9,3841,B00TA4OJVI,Wild Thing Wild Woman,Black Rainbow,https://images-na.ssl-images-amazon.com/images...,APPAREL,womens black rainbow cascading patch kimono ja...,99.99


## Text preprocessing

In [10]:
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
import re

def decontracted(phrase):
    # specific
    phrase = re.sub(r"won't", "will not", phrase)
    phrase = re.sub(r"can\'t", "can not", phrase)

    # general
    phrase = re.sub(r"n\'t", " not", phrase)
    phrase = re.sub(r"\'re", " are", phrase)
    phrase = re.sub(r"\'s", " is", phrase)
    phrase = re.sub(r"\'d", " would", phrase)
    phrase = re.sub(r"\'ll", " will", phrase)
    phrase = re.sub(r"\'t", " not", phrase)
    phrase = re.sub(r"\'ve", " have", phrase)
    phrase = re.sub(r"\'m", " am", phrase)
    phrase = re.sub(r"\n", "", phrase)
    return phrase

def remove_stopwords(text):
    '''this function will remove stopwords from text using nltk stopwords'''
    final_text = ''
    for word in text.split():
        if word not in stopwords.words('english'):
            final_text += word + ' '
    return final_text

def preprocess_title(text):
    # convert to lower case
    text = text.lower()
    # decontract
    text = decontracted(text)
    # remove all punctuations except a-z and c# and c++
    text = re.sub('[^a-z]+',' ',text)
    # remove stop words
    text = remove_stopwords(text)
    return text

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [ ]:
preprocessed_title = []
for text in tqdm(dff['title']):
    preprocessed_title.append(preprocess_title(text))

# Download Glove_840B_300d

In [ ]:
!wget --header="Host: downloads.cs.stanford.edu" --header="User-Agent: Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/87.0.4280.88 Safari/537.36" --header="Accept: text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.9" --header="Accept-Language: en-IN,en-GB;q=0.9,en-US;q=0.8,en;q=0.7" --header="Cookie: _ga=GA1.2.1563687661.1610043308; _gid=GA1.2.1018818887.1610043308" --header="Connection: keep-alive" "http://downloads.cs.stanford.edu/nlp/data/glove.840B.300d.zip" -c -O 'CurlWget1133'

In [ ]:
!unzip 'glove_840B_300d'

Archive:  glove_840B_300d
  inflating: glove.840B.300d.txt     


##Load Glove model

In [ ]:
# function to load glove vectors
from tqdm import tqdm
def load_glove_model(glove_file):
    print("Loading Glove Model")
    f = open(glove_file, 'r')
    model = {}
    vector_size = 300
    for line in tqdm(f):
        split_line = line.split()
        word = " ".join(split_line[0:len(split_line) - vector_size])
        embedding = np.array([float(val) for val in split_line[-vector_size:]])
        model[word] = embedding
    print("Done.\n" + str(len(model)) + " words loaded!")
    return model

In [ ]:
glove_model = load_glove_model('glove.840B.300d.txt')

736it [00:00, 7353.70it/s]

Loading Glove Model


2196017it [04:09, 8806.69it/s]

Done.
2195895 words loaded!


In [ ]:
# get list of lists for every sentence words
words = []
for line in tqdm(preprocessed_title):
    words.append(line.split())

In [ ]:
len(words)

16042

In [ ]:
# get one single list of whole corpus words
Words = []
for line in tqdm(preprocessed_title):
    Words.extend(line.split())

In [ ]:
print("All the words in the courpus", len(Words))

Words = set(Words)

print("The unique words in the courpus", len(Words))

inter_words = set(glove_model.keys()).intersection(Words)

print("The number of words that are present in both glove vectors and our courpus",
      len(inter_words),"(",np.round(len(inter_words)/len(Words)*100,3),"%)")

All the words in the courpus 148653
The unique words in the courpus 11125
The number of words that are present in both glove vectors and our courpus 8204 ( 73.744 %)


In [ ]:
from tqdm.auto import tqdm

words_corpus = {}
words_glove = set(glove_model.keys())
for i in tqdm(Words):
    if i in words_glove:
        words_corpus[i] = glove_model[i]

print("word 2 vec length", len(words_corpus))

# stronging variables into pickle files python
joblib.dump(words_corpus,'pickle_files/words_corpus.pkl')


word 2 vec length 8204


['pickle_files/words_corpus.pkl']

In [ ]:
len(words_corpus)

8204

In [ ]:
# keys of word corpus
words_corpus_keys = words_corpus.keys()
len(words_corpus_keys)

8647

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [ ]:
tfidf = TfidfVectorizer()
# fit and transform on question content
tfidf.fit_transform(preprocessed_title)

<16042x11107 sparse matrix of type '<class 'numpy.float64'>'
	with 143956 stored elements in Compressed Sparse Row format>

In [ ]:
# save tfidf
joblib.dump(tfidf,'pickle_files/tfidf.pkl')

['pickle_files/tfidf.pkl']

In [ ]:
dictionary = dict(zip(tfidf.get_feature_names(), list(tfidf.idf_)))
tfidf_words = set(tfidf.get_feature_names())
print(' Total tfidf words are ==> ',len(tfidf_words))

 Total tfidf words are ==>  11107


### Get tfidf weighted W2V on whole corpus

In [ ]:
tfidf_w2v_vector = [] # the weighted w2v for each sentence is stored in this list
for sentence in tqdm(words): # for each sentence words list
    vector = np.zeros(300) # as word vectors length is 300
    tf_idf_weight =0; # num of words with a valid vector in the sentence
    for word in sentence : # for each word in a review/sentence
        if (word in words_corpus_keys ) and (word in tfidf_words):
            vec = words_corpus[word] # getting the vector for each word
            # multiply idf value of word with tf of word
            tf_idf = dictionary[word]*(sentence.count(word)/len(sentence))
            vector += (vec * tf_idf) # calculating tfidf weighted w2v
            tf_idf_weight += tf_idf
    if tf_idf_weight != 0:
        vector /= tf_idf_weight
    tfidf_w2v_vector.append(vector)

In [ ]:
# save into disk
joblib.dump(tfidf_w2v_vector,'pickle_files/tfidf_glove_vectors.pkl')

['pickle_files/tfidf_glove_vectors.pkl']

# final function to get top k similar result

In [27]:
# -------------get glove words from word corpus-------------------------
words_corpus = joblib.load('pickle_files/words_corpus.pkl')
glove_words = set(words_corpus.keys())

# --------------load tfidf------------------------------
tfidf = joblib.load('pickle_files/tfidf.pkl')
# get tfidf words
tfidf_words = set(tfidf.get_feature_names())
# dictionary of tfidf
dictionary = dict(zip(tfidf.get_feature_names(), list(tfidf.idf_)))

#-----------------load tfidf_w2v_vectors of corpus----------------
tfidf_w2v_vector = joblib.load('pickle_files/tfidf_glove_vectors.pkl')

def find_similarity(question,top_n):
    ''' This function will find top similar result for given query'''
    start = time.time()
    # initialize  vector for user query
    main_vec = np.zeros(300)
    # initialize tfidf weight
    weight_sum = 0
    # preprocess question
    text = preprocess_title(question)
    #splitting the sentence
    text_list = list(text.split())
    for word in text_list:
        #finding if word is present in tfidf and in w2v words
        if word in tfidf_words and word in glove_words :
            #finding vector of word from glove model
            vect = words_corpus[word]
            #compute tfidf
            tf_idf = dictionary[word]*(text_list.count(word)/len(text_list)) 
            # adding vector * tfidf to main_vec
            main_vec+= (vect*tf_idf)
            # summing tfidf values
            weight_sum += tf_idf
    if weight_sum !=0:
        # devide by weight_sum
        main_vec /= weight_sum
    # find cosine similarity
    similarities =  cosine_similarity((main_vec).reshape(1, -1), Y=tfidf_w2v_vector, dense_output=True)
    # sort similarities 
    #print(similarities[0])
    
    sort = np.argsort(similarities[0])
    # get top similarity indices  in descending order
    similarity_index = np.array(list(reversed(sort)))
    # finad top n similarities
    top_similarity_index = similarity_index[1:top_n+1]
    # print top similarity values
    print('Top cosine similarities are ======>',similarities[0][top_similarity_index])
    # get original title of similar questions
    similar_questions = dff.iloc[top_similarity_index]
    #print(similar_questions)
    
    total_time = (time.time() - start)
    print('Total time ===========> ',total_time)
    return similar_questions,top_similarity_index


In [28]:
dff['title'][0]

'featherlite ladies long sleeve stain resistant tapered twill shirt 2xl onyx black stone '

In [33]:
title = 'kingyuan 105d 42e 46d 48c wider tear drop oval silicone breast forms breast enhancers fake boobs without adhesive layer'
top_10,top_10_id = find_similarity(title,10)

Top cosine similarities are ======> [1.         1.         0.99577173 0.99577173 0.99577173 0.99577173
 0.99577173 0.99236037 0.99236037 0.99236037]
Total time ===========>  0.06833958625793457


In [34]:
top_10_id

array([6948, 6637, 3603, 2226, 4199, 3291, 4794, 6682, 5412, 5331])

In [36]:
top_10

,asin,brand,color,medium_image_url,product_type_name,title,formatted_price
6948,B07213TY3D,KingYuan,Flesh,https://images-na.ssl-images-amazon.com/images...,SHIRT,kingyuan 75d 34c 36b 38a wider tear drop ova...,39.99
6637,B07213B45W,KingYuan,Flesh,https://images-na.ssl-images-amazon.com/images...,SHIRT,kingyuan 70d 34b 36a 38aa wider tear drop ov...,32.99
3603,B071WNSBT6,KingYuan,Flesh,https://images-na.ssl-images-amazon.com/images...,SHIRT,kingyuan 85d 36dd 38d 40c wider tear drop ov...,56.99
2226,B071WNMPQX,KingYuan,Flesh,https://images-na.ssl-images-amazon.com/images...,SHIRT,kingyuan 110d 44e 46dd 48d wider tear drop ov...,115.99
4199,B071JR3XX2,KingYuan,Flesh,https://images-na.ssl-images-amazon.com/images...,SHIRT,kingyuan 90d 38dd 40d 36e wider tear drop ov...,66.99
3291,B071JR3TVN,KingYuan,Flesh,https://images-na.ssl-images-amazon.com/images...,SHIRT,kingyuan 80d 34dd 36d 38c wider tear drop ov...,46.99
4794,B0721376PR,KingYuan,Flesh,https://images-na.ssl-images-amazon.com/images...,SHIRT,kingyuan 95d 40dd 42d 44c wider tear drop ov...,76.99
6682,B071G51ZN5,KingYuan,Flesh,https://images-na.ssl-images-amazon.com/images...,SHIRT,kingyuan 120d 48e 50dd 52d wider tear drop o...,143.99
5412,B071JR2MCH,KingYuan,Flesh,https://images-na.ssl-images-amazon.com/images...,SHIRT,kingyuan 100d 42dd 44d 46c wider tear drop o...,86.99
5331,B071JQM2PZ,KingYuan,Flesh,https://images-na.ssl-images-amazon.com/images...,SHIRT,kingyuan 115d 46e 48dd 50d wider tear drop o...,129.99


In [ ]:
title = 'womens unique 100 cotton  special olympics world games 2015 white size l '
top_10 = find_similarity(title,10)

Total time ===========>  0.10569095611572266


In [ ]:
top_10

,asin,brand,color,medium_image_url,product_type_name,title,formatted_price
1,B012YX2ZPI,HX-Kingdom Fashion T-shirts,White,https://images-na.ssl-images-amazon.com/images...,SHIRT,womens unique 100 cotton special olympics wor...,9.99
84,B012YX3I5E,HX-Kingdom Fashion T-shirts,Red,https://images-na.ssl-images-amazon.com/images...,SHIRT,womens new 100 cotton special olympics world ...,9.99
68,B012YX26Q6,HX-Kingdom Fashion T-shirts,Pink,https://images-na.ssl-images-amazon.com/images...,SHIRT,womens fashion tshirts special olympics world...,9.99
99,B012YX3XTU,HX-Kingdom Fashion T-shirts,Red,https://images-na.ssl-images-amazon.com/images...,SHIRT,womens casual tshirt special olympics world g...,9.99
46,B012YX2M3I,HX-Kingdom Fashion T-shirts,White,https://images-na.ssl-images-amazon.com/images...,SHIRT,womens best graphic tshirts special olympics ...,9.99
13840,B01338MTHM,MAM2 Arts,Black,https://images-na.ssl-images-amazon.com/images...,BOOKS_1973_AND_LATER,funny special olympics world sports games 2015...,9.50
162,B012YX2OJU,HX-Kingdom Fashion T-shirts,Skyblue,https://images-na.ssl-images-amazon.com/images...,SHIRT,womens new style tshirt special olympics worl...,9.99
27,B012YX4TVG,HX-Kingdom Fashion T-shirts,Royalblue,https://images-na.ssl-images-amazon.com/images...,SHIRT,womens unique tshirt special olympics world g...,9.99
231,B012YX4LVE,HX-Kingdom Fashion T-shirts,Orange,https://images-na.ssl-images-amazon.com/images...,SHIRT,womens vintage tshirt special olympics world ...,9.99
11060,B01338LWAM,MAM2 Arts,Skyblue,https://images-na.ssl-images-amazon.com/images...,BOOKS_1973_AND_LATER,cool special olympics world sports games 2015 ...,9.38
